In [33]:
import numpy as np 
import pandas as pd 
from keras import layers
from keras import models
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from os import listdir, makedirs
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionResNetV2
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers, regularizers
from keras import losses

In [8]:
RESOLUTION = 197
BATCH_SIZE = 20

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

train_generator = train_datagen.flow_from_directory(
        "../input/state-farm-distracted-driver-detection/train/",
        target_size=(RESOLUTION, RESOLUTION),
        batch_size=BATCH_SIZE,
        class_mode='categorical', subset="training")

val_generator = val_datagen.flow_from_directory(
        "../input/state-farm-distracted-driver-detection/train/",
        target_size=(RESOLUTION, RESOLUTION),
        batch_size=BATCH_SIZE,
        class_mode='categorical', subset="validation")

Found 15702 images belonging to 10 classes.
Found 6722 images belonging to 10 classes.


In [26]:
path = "../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
conv_base = InceptionResNetV2(include_top=False, input_shape=(RESOLUTION, RESOLUTION, 3), weights=path)

In [34]:
def inceptionv2_pretrained_model(model_inception, dropout_=False, regularizer_=False, regularizer_weight=0.001):
    model = Sequential()
    model.add(model_inception)
    model.add(Flatten())
    if dropout_:
        model.add(Dropout(0.5))
    if regularizer_:
        model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=regularizer_weight,
                                                                                            l2=regularizer_weight)))
    else:
        model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.summary()
    model.compile(loss=losses.poisson, optimizer=optimizers.Nadam(lr=1e-5), metrics=['acc'])

    return model

In [35]:
model = inceptionv2_pretrained_model(conv_base, dropout_=False, regularizer_=False, regularizer_weight=0.001)

TypeError: The added layer must be an instance of class Layer. Found: <tensorflow.python.keras.engine.training.Model object at 0x7fecd5e90710>

In [30]:
N_TRAIN = 15702
N_VAL = 6722
history = model.fit_generator(
        train_generator,
        steps_per_epoch=(N_TRAIN // BATCH_SIZE),
        epochs=20,
        validation_data=val_generator,
        validation_steps=(N_VAL // BATCH_SIZE) 
    )

NameError: name 'model' is not defined

In [31]:
history.params

NameError: name 'history' is not defined

In [32]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend(loc='best')

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc='best')

NameError: name 'history' is not defined